In [84]:
#!{sys.executable} -m pip install python-binance
#!pip install nomics-python
#!pip install ipynb
#!pip install python-ipynb

import sys
import numpy as np
import requests
import time
import dateparser
import pytz
import json
import os, os.path
import errno
import random
#!pip install import-ipynb
import import_ipynb
%run test_my_function.ipynb
#%run test_my_function.ipynb
#from test_my_function import *
#from ipynb.fs.defs.test_my_function import hello
#from test_my_function import *


from datetime import datetime
from binance.client import Client


api_key = '1mqe2khgk2qJAwpL4u2GQHm0pXrOoc0S2heu8Lgn51fH7UBZedLxPCVv56GMnABj'
api_secret = 'EPRUJORf6NVdGPL4tLsuRnhVpgcj55M7n1iyy5tzczYRxf8FyOC4DSAFQLLb8UHX'
client = Client(api_key, api_secret, tld='us')
list_of_symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT','BNBBTC', 'ETHBTC']

Variable a is 9.0


## Get and Save Clines
get historical klines from binance data and save it to a file
this is so we dont have to consantly call the api when testing 


In [7]:
def get_historical_klines(symbol, interval, start_str, end_str=None):
    """Get Historical Klines from Binance

    See dateparse docs for valid start and end string formats http://dateparser.readthedocs.io/en/latest/

    If using offset strings for dates add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"

    :param symbol: Name of symbol pair e.g BNBBTC
    :type symbol: str
    :param interval: Biannce Kline interval
    :type interval: str
    :param start_str: Start date string in UTC format
    :type start_str: str
    :param end_str: optional - end date string in UTC format
    :type end_str: str

    :return: list of OHLCV values

    """
    # create the Binance client, no need for api key

    # init our list
    output_data = []

    # setup the max limit
    limit = 500

    # convert interval to useful value in seconds
    timeframe = interval_to_milliseconds(interval)

    # convert our date strings to milliseconds
    start_ts = date_to_milliseconds(start_str)

    # if an end time was passed convert it
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)

    idx = 0
    # it can be difficult to know when a symbol was listed on Binance so allow start time to be before list date
    symbol_existed = False
    while True:
        # fetch the klines from start_ts up to max 500 entries or the end_ts if set
        temp_data = client.get_klines(
            symbol=symbol,
            interval=interval,
            limit=limit,
            startTime=start_ts,
            endTime=end_ts
        )

        # handle the case where our start date is before the symbol pair listed on Binance
        if not symbol_existed and len(temp_data):
            symbol_existed = True

        if symbol_existed:
            # append this loops data to our output data
            output_data += temp_data

            # update our start timestamp using the last value in the array and add the interval timeframe
            start_ts = temp_data[len(temp_data) - 1][0] + timeframe
        else:
            # it wasn't listed yet, increment our start date
            start_ts += timeframe

        idx += 1
        # check if we received less than the required limit and exit the loop
        if len(temp_data) < limit:
            # exit the while loop
            break

        # sleep after every 3rd call to be kind to the API
        if idx % 3 == 0:
            time.sleep(1)

    return output_data

def interval_to_milliseconds(interval):
    """Convert a Binance interval string to milliseconds

    :param interval: Binance interval string 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w
    :type interval: str

    :return:
         None if unit not one of m, h, d or w
         None if string not in correct format
         int value of interval in milliseconds
    """
    ms = None
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60
    }

    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms

def date_to_milliseconds(date_str):
    """Convert UTC date to milliseconds

    If using offset strings add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"

    See dateparse docs for formats http://dateparser.readthedocs.io/en/latest/

    :param date_str: date in readable format, i.e. "January 01, 2018", "11 hours ago UTC", "now UTC"
    :type date_str: str
    """
    # get epoch value in UTC
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    # parse our date string
    d = dateparser.parse(date_str)
    # if the date is not timezone aware apply UTC timezone
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)

    # return the difference in time
    return int((d - epoch).total_seconds() * 1000.0)

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise


def save_historic_klines_datafile(symbol, start, end, interval):
    #Collects kline historical data , output and saves to file
    klines = get_historical_klines(symbol, interval, start, end)

    # open a file with filename including symbol, interval and start and end converted to milliseconds
    mkdir_p("historicData\Binance_{}".format(symbol))
    with open(
        "historicData\Binance_{}\{}_{}-{}.json".format(
            symbol,
            interval,
            start,
            end,

        ),
        "w+"  # set file write mode
    ) as f:
        f.write(json.dumps(klines))
        f.write("\n")
 

# Save and read candlestick from the file 

In [8]:
def saveCurrentCurrencyCandleStickToFile():
    # get the candle stick version of the graph

    # get system status
    # client.ping()

    klines = [[] * 5]
    # Step 1: read in data from the .txt files
    # they will be stored in multipple 2d numpy arrays
    for trade in market_list:

        # get chosen candlestick information and send it to the file
        try:
            save_historic_klines_datafile(trade, "1 hours ago UTC", "now UTC", Client.KLINE_INTERVAL_5MINUTE)
            save_historic_klines_datafile(trade, "8 hours ago UTC", "now UTC", Client.KLINE_INTERVAL_15MINUTE)
            save_historic_klines_datafile(trade, "1 week ago UTC", "now UTC", Client.KLINE_INTERVAL_8HOUR)
            save_historic_klines_datafile(trade, "1 month ago UTC", "now UTC", Client.KLINE_INTERVAL_1DAY)
            save_historic_klines_datafile(trade, "3 months ago UTC", "now UTC", Client.KLINE_INTERVAL_3DAY)
            save_historic_klines_datafile(trade, "12 months ago UTC", "now UTC", Client.KLINE_INTERVAL_1WEEK)
        except():
            pass
    # exchanges and candlesticks are saved in historicData as data files
    # get the values and put them into a numpy array
    """ thiis is what is stored as a kline in each file
    1499040000000,  # Open time
    "0.01634790",  # Open
    "0.80000000",  # High
    "0.01575800",  # Low
    "0.01577100",  # Close
    "148976.11427815",  # Volume
    1499644799999,  # Close time
    "2434.19055334",  # Quote asset volume
    308,  # Number of trades
    "1756.87402397",  # Taker buy base asset volume
    "28.46694368",  # Taker buy quote asset volume
    "17928899.62484339"  # Can be ignored"""





def load_data(path):
    with open(path) as f:
        data = json.load(f)
    #print(data)
    return np.asarray(data)

def readData(klines):
    with open('Klines_File_List.txt', 'w') as f:
        # look at Shdr_saveHistorical_Klines line 1
        for trade in market_list:

            # we have data for one of the items in the market list
            # put kline info into file to be used


            for i in klines:
                time1 = int(klines[i][0])
                open1 = float(klines[i][1])
                Low = float(klines[i][2])
                High = float(klines[i][3])
                Close = float(klines[i][4])
                Volume = float(klines[i][5])
                format_kline = "{}, {}, {}, {}, {}, {}\n".format(time1, open1, High, Low, Close, Volume)
                f.write(format_kline)


    """
    #Matplotlib visualization how-to from: https://pythonprogramming.net/candlestick-ohlc-graph-matplotlib-tutorial/
    fig, ax = plt.subplots()
    Shdr.mpl_finance.candlestick_ochl(ax, ochl, width=1)
    plt.plot(time3, three_period_moving_ave, color='green', label='3 Period MA - Open')
    plt.plot(time10, ten_period_moving_ave, color='blue', label='10 Period MA - Open')
    #ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d-%h-%m')) #Converting to date format not working
    ax.set(xlabel='Date', ylabel='Price', title='{} {}-{}'.format(symbol, start, end))
    plt.legend()
    plt.show()
    """
def saveTestDataToFile():

    for trade in market_list:
        #year
        for a in range(0, 2):
            #month
            for b in range(0, 5):
                #weeeks
                for c in range(0, 4):
                    #hours
                    for d in range(0, 4):
                        # get chosen candlestick information and send it to the file
                        #print(Shdr.date_to_milliseconds("1 years 1 months 1 days 1 hours ago UTC"))
                        hours=d*4 #every 4 hours
                        months=b*2 # every 2 months
                        end_time = "{} years {} months {} weeks {} hours ago UTC".format(a, months, c, hours)
                        try:
                            #one hour
                            start_time_1_hour = "{} years {} months {} weeks {} hours ago UTC".format(a, months, c, (hours + 1))

                            save_historic_klines_datafile(trade, start_time_1_hour, end_time, Client.KLINE_INTERVAL_5MINUTE)
                            # 8 hours
                            start_time_8_hour = "{} years {} months {} weeks {} hours ago UTC".format(a, months, c, (hours + 8))
                            save_historic_klines_datafile(trade, start_time_8_hour, end_time, Client.KLINE_INTERVAL_15MINUTE)
                            #one week
                            start_time_1_week = "{} years {} months {} weeks {} hours ago UTC".format(a, months, (c + 1), hours)
                            save_historic_klines_datafile(trade, start_time_1_week, end_time, Client.KLINE_INTERVAL_8HOUR)
                            #1 month
                            start_time_1_month = "{} years {} months {} weeks {} hours ago UTC".format(a, (months + 1), c, hours)
                            save_historic_klines_datafile(trade, start_time_1_month, end_time, Client.KLINE_INTERVAL_1DAY)
                            # 3 months
                            start_time_3_month = "{} years {} months {} weeks {} hours ago UTC".format(a, (months + 3), c, hours)
                            save_historic_klines_datafile(trade, start_time_3_month, end_time, Client.KLINE_INTERVAL_3DAY)
                            # 1 year
                            start_time_1_year = "{} years {} months {} weeks {} hours ago UTC".format((a + 1), months, c, hours)
                            save_historic_klines_datafile(trade, start_time_1_year, end_time, Client.KLINE_INTERVAL_1WEEK)
                        except():
                            pass
                        time.sleep(2)
                        # see saveCurrentCurrencyCandleStickToFile() for detail of what is saved to file



In [278]:
market_list = list_of_symbols
#saveCurrentCurrencyCandleStickToFile()
#saveTestDataToFile()


## reading for tensor 
read any given trade from the it designated file. Data will be stored in a ditionary:
 
 tradeData = {
        'tradeData_H' : {
            'kline': tradeData_H,
            'time3': time3H,
            'time10': time10H,     
            },
        'tradeData_HH' : {
            'kline': tradeData_HH,
            'time3': time3HH,
            'time10': time10HH,   
            },
        'tradeData_W' : {
            'kline': tradeData_W,
            'time3': time3W,
            'time10': time10W,
            },
        'tradeData_M' : {
            'kline': tradeData_M,
            'time3': time3M,
            'time10': time10M,  
            },
        'tradeData_MM' : {
            'kline': tradeData_MM,
            'time3': time3MM,
            'time10': time10MM,  
            },
        'tradeData_Y' : {
            'kline': tradeData_Y,
            'time3': time3Y,
            'time10': time10Y,  
            }
    }

## Run some Basic Calculations

run some basic trade history averages calculations for the algorithm for extra data t give it
this will be given back as an array whose length smaller than the that of trade data. time 3 averages 3 data points, time10 averaged 10 data points. 

In [9]:
"""read data does not open a file and return what it holds
we need to change that """
def getTFData(trade):
    tradeData_H = load_data(
        "historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_5MINUTE, "1 hours ago UTC",
                                                       "now UTC"))
    tradeData_HH = load_data(
        "historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_15MINUTE, "8 hours ago UTC",
                                                       "now UTC"))
    tradeData_W = load_data(
        "historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_8HOUR, "1 week ago UTC", "now UTC"))
    tradeData_M = load_data(
        "historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_1DAY, "1 month ago UTC", "now UTC"))
    tradeData_MM = load_data(
        "historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_3DAY, "3 months ago UTC",
                                                       "now UTC"))
    tradeData_Y = load_data(
        "historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_1WEEK, "12 months ago UTC",
                                                       "now UTC"))

    # SIMPLIFY TO GET CANDLESTICK DATA AND DIFFRENT TIME OFSET GRAPHS
    time3H, time10H = getMovingAverages(tradeData_H)
    time3HH, time10HH = getMovingAverages(tradeData_HH)
    time3W, time10W = getMovingAverages(tradeData_W)
    time3M, time10M = getMovingAverages(tradeData_M)
    time3MM, time10MM = getMovingAverages(tradeData_MM)
    time3Y, time10Y = getMovingAverages(tradeData_Y)
    
    tradeData = {
        'tradeData_H' : {
            'kline': tradeData_H,
            'time3': time3H,
            'time10': time10H,     
            },
        'tradeData_HH' : {
            'kline': tradeData_HH,
            'time3': time3HH,
            'time10': time10HH,   
            },
        'tradeData_W' : {
            'kline': tradeData_W,
            'time3': time3W,
            'time10': time10W,
            },
        'tradeData_M' : {
            'kline': tradeData_M,
            'time3': time3M,
            'time10': time10M,  
            },
        'tradeData_MM' : {
            'kline': tradeData_MM,
            'time3': time3MM,
            'time10': time10MM,  
            },
        'tradeData_Y' : {
            'kline': tradeData_Y,
            'time3': time3Y,
            'time10': time10Y,  
            }
    }
    """tradeData = {
        'tradeData_H' : tradeData_H,
        'tradeData_HH' : tradeData_HH,
        'tradeData_W' :  tradeData_W,
        'tradeData_M' : tradeData_M,
        'tradeData_MM' : tradeData_MM,
        'tradeData_Y' : tradeData_Y
        }"""
    return tradeData
    
def getMovingAverages(klines):

    ochl = []
    list_of_open = []
    three_period_moving_ave = []
    time3=[]
    five_period_moving_ave = []
    ten_period_moving_ave = []
    time10=[]

    for kline in klines:
        #print(kline)
        time1 = int(kline[0])
        open1 = float(kline[1])
        Low = float(kline[2])
        High = float(kline[3])
        Close = float(kline[4])
        Volume = float(kline[5])

        ochl.append([time1, open1, Close, High, Low, Volume])


        #track opening prices, use for calculating moving averages
        list_of_open.append(open1)
            #Calculate three 'period' moving average - Based on Candlestick duration
        if len(list_of_open)>2:
            price3=0
            for pri in list_of_open[-3:]:
                price3+=pri
            three_period_moving_ave.append(float(price3/3))
            time3.append(time1)
        #Perform Moving Average Calculation for 10 periods
        if len(list_of_open)>9:
            price10=0
            for pri in list_of_open[-10:]:
                price10+=pri
            ten_period_moving_ave.append(float(price10/10))
            time10.append(time1)
    return three_period_moving_ave,ten_period_moving_ave


In [280]:
print(getTFData('BTCUSDT'))

{'tradeData_H': {'kline': array([['1620405000000', '57603.96000000', '57785.14000000',
        '57603.96000000', '57723.34000000', '3.44241500',
        '1620405299999', '198771.66483212', '121', '2.54197900',
        '146781.19622163', '0'],
       ['1620405300000', '57713.70000000', '57751.40000000',
        '57538.00000000', '57626.10000000', '3.58773100',
        '1620405599999', '206612.94945406', '112', '0.95034200',
        '54736.20039402', '0'],
       ['1620405600000', '57649.99000000', '57776.42000000',
        '57589.08000000', '57729.02000000', '8.67998500',
        '1620405899999', '500933.65386495', '248', '5.38045400',
        '310534.59763046', '0'],
       ['1620405900000', '57709.16000000', '57859.03000000',
        '57661.60000000', '57821.44000000', '1.27053600',
        '1620406199999', '73410.50761637', '82', '0.49422500',
        '28549.21762140', '0'],
       ['1620406200000', '57824.54000000', '58077.62000000',
        '57824.54000000', '58077.62000000', '2.85

In [10]:
"""get all of the data from the files that we have created. we need the nested arrays to mirror function 'saveTestDataToFiles'
or else this will not work"""
def getTestDataFromFile():

    for trade in market_list:
        #year
        for a in range(0, 3):
            #month
            for b in range(0, 4):
                #weeeks
                for c in range(0, 4):
                    #hours
                    for d in range(0, 4):
                        # get chosen candlestick information and send it to the file
                        #print(Shdr.date_to_milliseconds("1 years 1 months 1 days 1 hours ago UTC"))
                        hours=d*4 #every 4 hours
                        months=b*2 # every 2 months
                        
                        get_klines_averages_datafile(trade, a, months, c, hours)
                        
                        
                        # see saveCurrentCurrencyCandleStickToFile() for detail of what is saved to file

def get_klines_averages_datafile(trade, a, months, c, hours):
     
    end_time = "{} years {} months {} weeks {} hours ago UTC".format(a, months, c, hours)
    try:
        #one hour
        start_time_1_hour = "{} years {} months {} weeks {} hours ago UTC".format(a, months, c, (hours + 1))
        tradeData_H = load_data( "historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_5MINUTE, 
                                                start_time_1_hour, end_time))
        #print("historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_5MINUTE, 
                                                #start_time_1_hour, end_time))
        # 8 hours
        start_time_8_hour = "{} years {} months {} weeks {} hours ago UTC".format(a, months, c, (hours + 8))
        tradeData_HH = load_data("historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_15MINUTE,
                                                start_time_8_hour, end_time))
        #one week
        start_time_1_week = "{} years {} months {} weeks {} hours ago UTC".format(a, months, (c + 1), hours)
        tradeData_W = load_data("historicData\Binance_{}\{}_{}-{}.json".format(trade,  Client.KLINE_INTERVAL_8HOUR,
                                                start_time_1_week, end_time))
        #1 month
        start_time_1_month = "{} years {} months {} weeks {} hours ago UTC".format(a, (months + 1), c, hours)
        tradeData_M = load_data("historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_1DAY,
                                                start_time_1_month, end_time))
        # 3 months
        start_time_3_month = "{} years {} months {} weeks {} hours ago UTC".format(a, (months + 3), c, hours)
        tradeData_MM = load_data("historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_3DAY,
                                                start_time_3_month, end_time))
        # 1 year
        start_time_1_year = "{} years {} months {} weeks {} hours ago UTC".format((a + 1), months, c, hours)
        tradeData_Y = load_data("historicData\Binance_{}\{}_{}-{}.json".format(trade, Client.KLINE_INTERVAL_1WEEK,
                                                start_time_1_year, end_time))
        
         # SIMPLIFY TO GET CANDLESTICK DATA AND DIFFRENT TIME OFSET GRAPHS
        time3H, time10H = getMovingAverages(tradeData_H)
        time3HH, time10HH = getMovingAverages(tradeData_HH)
        time3W, time10W = getMovingAverages(tradeData_W)
        time3M, time10M = getMovingAverages(tradeData_M)
        time3MM, time10MM = getMovingAverages(tradeData_MM)
        time3Y, time10Y = getMovingAverages(tradeData_Y)

        tradeData = {
            'tradeData_H' : {
                'kline': tradeData_H,
                'time3': time3H,
                'time10': time10H,     
                },
            'tradeData_HH' : {
                'kline': tradeData_HH,
                'time3': time3HH,
                'time10': time10HH,   
                },
            'tradeData_W' : {
                'kline': tradeData_W,
                'time3': time3W,
                'time10': time10W,
                },
            'tradeData_M' : {
                'kline': tradeData_M,
                'time3': time3M,
                'time10': time10M,  
                },
            'tradeData_MM' : {
                'kline': tradeData_MM,
                'time3': time3MM,
                'time10': time10MM,  
                },
            'tradeData_Y' : {
                'kline': tradeData_Y,
                'time3': time3Y,
                'time10': time10Y,  
                }
        }
        return tradeData
    
    except():
        pass
   
    
   
    

In [390]:
get_klines_averages_datafile('BTCUSDT', 0, 4, 1, 4)

{'tradeData_H': {'kline': array([['1609417800000', '28838.98000000', '28922.46000000',
          '28825.82000000', '28874.26000000', '0.60984600',
          '1609418099999', '17612.12339869', '56', '0.12328100',
          '3558.54612665', '0'],
         ['1609418100000', '28890.34000000', '28944.17000000',
          '28890.34000000', '28907.42000000', '1.59267000',
          '1609418399999', '46081.65882387', '42', '1.40592700',
          '40680.31407899', '0'],
         ['1609418400000', '28910.15000000', '28943.10000000',
          '28910.15000000', '28914.63000000', '0.09716700',
          '1609418699999', '2810.91810314', '28', '0.02886900',
          '834.95098932', '0'],
         ['1609418700000', '28900.42000000', '28900.42000000',
          '28683.97000000', '28775.90000000', '3.54885200',
          '1609418999999', '102160.73599269', '106', '1.87884900',
          '54107.06661063', '0'],
         ['1609419000000', '28795.19000000', '28823.41000000',
          '28674.95000000',

## basic radom setup

buy or sell randomly based on random market pair from market list
has higher chance of 0 than any other position. 0 happens if there is an error or just as another option
we will get the current price at the 10 point avereage for the time closest to current moment in time

In [11]:
def buySellBasic(trade, klineInfo):
    #buy sell or do nothing
    case = random.randrange(3)
    #get the current exhange for what we are trying to buy/sell
    #getAverage(klineInfo)
    #buy 
    price = getCurrentPrice(klineInfo)
    if price == None:
        price = 0
    #buy 
    if case == 0:
        return price
    #sell
    elif case == 1:
        return price*(-1) 
    #pass
    else:
        return 0
    
def getCurrentPrice(klineInfo):
    
    if klineInfo['tradeData_H']:
        if klineInfo['tradeData_H']['time10']:
            if klineInfo['tradeData_H']['time10'][2] :
                #print(klineInfo)
                return klineInfo['tradeData_H']['time10'][2]
    else:
        return 0

In [15]:
trade = 'BTCUSDT'
klineInfo = get_klines_averages_datafile(trade, 1, 4, 1, 4)
buySellBasic(trade, klineInfo)

0

## Simulation

In [12]:
def runSimulation():
    
    accounts = {'BTC':1000,'USDT':1000,'ETH':1000,'BNB':1000}
    #print(accounts)
    #for trade in market_list:
    #year
    
    
    for a in range(1, -1, -1):
        #pick a random trade in the simulation
        #month
        #print(a)
        for b in range(3, -1, -1):
            #weeeks
            #print('weeks: '+str(b))
            for c in range(3, -1, -1):
                #hours
                #print('hours'+c)
                for d in range(3, -1, -1):
                    
                    trade = random.choice(market_list)
                    
                    # get chosen candlestick information and send it to the file
                    #print(Shdr.date_to_milliseconds("1 years 1 months 1 days 1 hours ago UTC"))
                    hours=d*4 #every 4 hours
                    months=b*2 # every 2 months
                    
                    klineInfo = get_klines_averages_datafile(trade, a, months, c, hours)
                    price = buySellBasic(trade, klineInfo)
                    #print(price)
                    if (price is not None and  price is not 0):
                        
                        # see saveCurrentCurrencyCandleStickToFile() for detail of what is saved to file    
                        if trade == 'BTCUSDT':
                            accounts = priceCalc('BTC','USDT',price,accounts)
                        elif trade == 'ETHUSDT':
                             accounts = priceCalc('ETH','USDT',price,accounts)
                        elif trade == 'BNBUSDT':
                            acounts = priceCalc('BNB','USDT',price,accounts)
                        elif trade == 'BNBBTC':
                            accounts = priceCalc('BNB','BTC',price,accounts)
                        elif trade == 'ETHBTC':
                            accounts = priceCalc('ETH','BTC',price,accounts)
                        else:
                            pass
                    #print(trade, end= ' ')
                    #print(accounts)
        #print(accounts)
        return accounts
    
    
def priceCalc(one , two, price, accounts):
    #model: onetwo 
    #buy BTC sell UDST
    #print(price)
    if price>0:
        accounts[two] -= 10 
        #print('1: ', end = '')
        accounts[one] += 10/price
    #sell BTC buy USDT 
    if price<0:
        accounts[two] -= price*10
        #print('2: ', end = '')
        accounts[one] -= 10
    #print(price)
    return accounts


In [409]:
simulationResults=runSimulation()
print(simulationResults)

{'BTC': 871.5060259497934, 'USDT': 190406.81810000003, 'ETH': 3707.3243194361903, 'BNB': 25605.3331957404}


In [410]:
btc = getCurrentPrice(get_klines_averages_datafile('BTCUSDT', 0, 0, 0, 0))
bnb = getCurrentPrice(get_klines_averages_datafile('BNBUSDT', 0, 0, 0, 0))
eth = getCurrentPrice(get_klines_averages_datafile('ETHUSDT', 0, 0, 0, 0))
print(simulationResults)
final_USDT = simulationResults['BTC']*btc+simulationResults['BNB']*bnb+simulationResults['ETH']*eth+simulationResults['USDT']
print(final_USDT)

{'BTC': 871.5060259497934, 'USDT': 190406.81810000003, 'ETH': 3707.3243194361903, 'BNB': 25605.3331957404}
80423693.45318475


## Hooray

we have finished creating our simulation. we can now make random trades.
using this game with set rules, we should make a nural network that wants to end up with thegreatest total final profit

get the dimensions of our array

In [85]:
klineInfo=get_klines_averages_datafile('BTCUSDT', 0, 0, 0, 0)
print('x:',len(klineInfo))
for i in klineInfo:
    print('\ny:',len(klineInfo[i]), end = ' ')
    for j in klineInfo[i]:
        print('\tz:',len(klineInfo[i][j]))
hello()

x: 6

y: 3 	z: 12
	z: 10
	z: 3

y: 3 	z: 32
	z: 30
	z: 23

y: 3 	z: 21
	z: 19
	z: 12

y: 3 	z: 30
	z: 28
	z: 21

y: 3 	z: 29
	z: 27
	z: 20

y: 3 	z: 52
	z: 50
	z: 43


'hello'